In [1]:
import os
import os.path as path
import re
from enum import Enum
from functools import total_ordering

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
import matplotlib.patheffects as PathEffects
import pandas as pd
import seaborn as sns

from tqdm.notebook import tqdm

mpl.style.use('seaborn')

In [2]:
subjPattern = re.compile(r"^\d+$")
trialPattern = re.compile(r"^(?P<block>\d\d)_(?P<trial>\d\d)(?P<data>\w+)\.tsv$")

ID01 = "00"
ID02 = "01"
ID03 = "02"
IDENTS = [ID01, ID02, ID03]

subjects = [d for d in os.listdir() if subjPattern.match(d)]
print(', '.join(subjects))

009, 011, 004, 007, 012, 018, 005, 020, 019, 015, 013, 022, 014, 023, 010, 016, 001, 021, 003, 008, 006, 017, 002


In [3]:
@total_ordering
class Boxes(Enum):
    NoBox  = 0
    SmallL = 10
    SmallC = 11
    SmallR = 12
    LargLC = 20
    LargCR = 21
    LargLR = 22

    def __lt__(self, other):
        if self.__class__ is other.__class__:
            return self.value < other.value
        return NotImplemented

    def __int__(self):
        return self.value

# w x d x h
smBox = [.6, .3, .9]
lgBox = [.95, .3, 1.8]
pRadius = .225
hwWidth = 2.85
segmentLength = 2
hwStartX = [0, 4, 8]

hallway1 = [
            Boxes.SmallC,
            Boxes.SmallC,
            Boxes.LargCR,
            Boxes.SmallL,
            Boxes.SmallR,
            Boxes.SmallC,
            Boxes.LargLR,
            Boxes.SmallL,
            Boxes.SmallL,
            Boxes.LargLC, # transition to 5
            Boxes.SmallC,
            Boxes.SmallR,
            Boxes.LargLR,
            Boxes.SmallC,
            Boxes.SmallL,
            Boxes.SmallR,
            Boxes.LargCR,
            Boxes.SmallC,
            Boxes.SmallC
]

hallway2 = [
            Boxes.SmallL,
            Boxes.SmallC,
            Boxes.LargCR,
            Boxes.SmallL,
            Boxes.SmallL,
            Boxes.SmallR,
            Boxes.LargLC,
            Boxes.SmallR,
            Boxes.SmallL,
            Boxes.LargCR, # transition to 3
            Boxes.SmallR,
            Boxes.SmallL,
            Boxes.LargLR,
            Boxes.SmallC,
            Boxes.SmallL,
            Boxes.SmallC,
            Boxes.LargCR,
            Boxes.SmallC,
            Boxes.SmallL
]

hallway3 = [
            Boxes.SmallL,
            Boxes.SmallC,
            Boxes.LargCR,
            Boxes.SmallR,
            Boxes.SmallL,
            Boxes.SmallC,
            Boxes.LargLR,
            Boxes.SmallL,
            Boxes.SmallC,
            Boxes.LargLC, # transition to 7
            Boxes.SmallR,
            Boxes.SmallR,
            Boxes.LargLC,
            Boxes.SmallC,
            Boxes.SmallR,
            Boxes.SmallL,
            Boxes.LargCR,
            Boxes.SmallC,
            Boxes.SmallL
]

hallways = [hallway1, hallway2, hallway3]


In [4]:
def load_subject(subjId):
    datadict = {
        ID01 : {ID01 : [], ID02 : [], ID03 : []}, # block 1
        ID02 : {ID01 : [], ID02 : [], ID03 : []}, # block 2
        ID03 : {ID01 : [], ID02 : [], ID03 : []} # block 3
    }
    files = os.listdir(subjId)
    for file in files:
        match = trialPattern.match(file)
        if not match:
            print("Error!", path.join(subjId, file))
            continue
        block_id = match.groups()[0]
        trial_id = match.groups()[1]
        datadict[block_id][trial_id].append(file)
        
    return datadict

def load_csv(filepath:str=None, subject:str=None, file:str=None) -> pd.DataFrame:
    if filepath is None:
        filepath = path.join(subject, file)
    df = pd.read_csv(filepath, sep='\t', usecols=lambda c: not c.startswith('Unnamed:'))
    return df


def vec2coord(s:pd.Series, keep=[0,1,2,3]):
    """Convert the stored vector in str: '(x,y,z)' to df with columns x&y"""
    vecs = s.apply(lambda v: np.array(v[1:-1].split(', ')).astype(float))
    p = pd.DataFrame(np.stack(vecs.values))
    if len(keep) > len(p.columns):
        keep = range(len(p.columns))
    labels = np.array(['x', 'y', 'z', 'w'])
    p = p[p.columns[keep]]
    p.columns = labels[:len(keep)]
    p.index = s.index
    return p

def pos2path(s:pd.Series, hallwayId:int):
    # convert text vector to 2d vector array
    # [2,0] "rotates" vector, hallways are drawn left -> right
    # so x (left, right) becomes y, and z (forward) becomes x
    p = vec2coord(s, [2,0])
    # 0-center path by subtracting hallway offset (id * 4)
    # in Unity right is positive, left is negative
    # we must negate to get correct orientation
    # then add half-width to put correct starting position
    p.y = -1 * (p.y - hallwayId * 4) + hwWidth / 2
    # add segment-length to account for first empty room behind start
    p.x += segmentLength
    return p

def round_to_multiple(number, multiple):
    return multiple * round(number / multiple)

In [6]:
def pos2dist(pos_data:pd.Series, ref_data:pd.Series=None) -> pd.Series:
    """Calculate distance travelled by subject"""
    pos = vec2coord(pos_data)
    if ref_data is not None:
        ref = vec2coord(ref_data)
        pos = pos - ref
    return np.sqrt(((pos.shift() - pos)**2).sum(axis=1))

def quaternions2anglediff(s:pd.Series) -> pd.Series:
    """Calculate angle difference between successive quaternions"""
    rot = vec2coord(s)
    dot = (rot.shift() * rot).sum(axis=1, min_count=3).abs()
    # min(dot, 1)
    dot[dot > 1] = 1
    # if dot product is 1, quaternions are equal, and thus 0 angle
    dot[dot > 1 - 1e-5] = 0
    # otherwise angle in radians is arccos(dot) * 2; convert to degrees
    dot[dot != 0] = np.arccos(dot[dot != 0]) * 2.0 * 180 / np.pi
    # 0 out first measurement
    dot.iloc[0] = 0

    return dot

all_subjects = []
for subj in tqdm(subjects):
    filemapping = load_subject(subj)
    blocks_dfs = []
    for block, trialfiles in filemapping.items():
        trial_dfs = []
        block_keys = []
        for trial, files in trialfiles.items():
            files = sorted(files)
            condition = load_csv(subject=subj, file=files[-1]).loc[0, 'GazeCondition']
            cond_name = "G.Ignored" if condition == "GazeIgnored" else "G.Locked" if condition == "SimulationFixedToGaze" else "G.Assisted"
            
            # load head data as vectors
            dfEng = load_csv(subject=subj, file=files[0])
            dfEng.set_index("TimeStamp", inplace=True)

            head = []
            # calculate distance moved
            head.append(pos2dist(dfEng['XRHeadPos'], dfEng['XROriginPos']))
            head.append(quaternions2anglediff(dfEng['XRHeadRot']))
            head = pd.concat(head, axis=1, keys=['dMove', 'dAngle'])

            # load left hand
            handL = []
            handL.append(pos2dist(dfEng['HandLPos']))
            handL.append(quaternions2anglediff(dfEng['HandLRot']))
            handL.append(dfEng['HandLInBox'])
            handL.append(dfEng['HandLInWall'])
            handL = pd.concat(handL, axis=1, keys=['dMove', 'dAngle', 'InBox', 'InWall'])

            # load right hand
            handR = []
            handR.append(pos2dist(dfEng['HandRPos']))
            handR.append(quaternions2anglediff(dfEng['HandRRot']))
            handR.append(dfEng['HandRInBox'])
            handR.append(dfEng['HandRInWall'])
            handR = pd.concat(handR, axis=1, keys=['dMove', 'dAngle', 'InBox', 'InWall'])

            trial_df = pd.concat([head, handL, handR], axis=1, keys=['Head', 'LHand', 'RHand'])

            trial_dfs.append(trial_df)
            block_keys.append(cond_name)
        
        blocks_dfs.append(pd.concat(trial_dfs, axis=0, keys=block_keys, names=['Condition'] + trial_dfs[0].index.names))
    
    # concatenate all blocks
    df = pd.concat(blocks_dfs, axis=0, keys=IDENTS, names=['Block'] + blocks_dfs[0].index.names)
    all_subjects.append(df)

df = pd.concat(all_subjects, axis=0, keys=subjects, names=['Subject'] + all_subjects[0].index.names)

  0%|          | 0/23 [00:00<?, ?it/s]

Error! 018/01_02EngineDataRecord.tsv.bckp
Error! 023/01_01SingleEyeDataRecordR.tsv.bckp
Error! 021/01_02EngineDataRecord.tsv.bckp


In [7]:
df.to_hdf('data.h5', key='headmovement', complevel=7)
df

Head            LHand  \
                                                dMove dAngle     dMove   
Subject Block Condition  TimeStamp                                       
009     00    G.Assisted 637934834746343441  0.000000    0.0  0.000000   
                         637934834746363547  0.000000    0.0  0.000000   
                         637934834746713438  0.000807    0.0  0.011723   
                         637934834746713438  0.000000    0.0  0.000000   
                         637934834746853421  0.001031    0.0  0.017365   
...                                               ...    ...       ...   
002     02    G.Ignored  637933183429876406  0.000168    0.0  0.000202   
                         637933183430046390  0.000132    0.0  0.000145   
                         637933183430226401  0.000283    0.0  0.000517   
                         637933183430441470  0.000324    0.0  0.000262   
                         637933183430621610  0.000081    0.0  0.000158   

                                                                        RHand  \
                                               dAngle  InBox InWall     dMove   
Subject Block Condition  TimeStamp                                              
009     00    G.Assisted 637934834746343441  0.000000  False  False  0.000000   
                         637934834746363547  0.000000  False  False  0.000000   
                         637934834746713438  0.651677  False  False  0.009960   
                         637934834746713438  0.000000  False  False  0.000000   
                         637934834746853421  1.082404  False  False  0.013440   
...                                               ...    ...    ...       ...   
002     02    G.Ignored  637933183429876406  0.000000  False  False  0.000215   
                         637933183430046390  0.000000  False  False  0.000063   
                         637933183430226401  0.000000  False  False  0.000022   
                         637933183430441470  0.000000  False  False  0.000243   
                         637933183430621610  0.000000  False  False  0.000238   

                                                                     
                                               dAngle  InBox InWall  
Subject Block Condition  TimeStamp                                   
009     00    G.Assisted 637934834746343441  0.000000  False  False  
                         637934834746363547  0.000000  False  False  
                         637934834746713438  4.549488  False  False  
                         637934834746713438  0.000000  False  False  
                         637934834746853421  4.366576  False  False  
...                                               ...    ...    ...  
002     02    G.Ignored  637933183429876406  0.000000  False  False  
                         637933183430046390  0.000000  False  False  
                         637933183430226401  0.000000  False  False  
                         637933183430441470  0.000000  False  False  
                         637933183430621610  0.000000  False  False  

[1128611 rows x 10 columns]

In [31]:
summed = df.groupby(df.index.names[:-1]).sum()
zscoredHeadMove = []
for subj in tqdm(subjects):
    s = summed.loc[subj, slice(ID02, ID03), :, :]
    zscored = s.apply(lambda c: (c - c.mean()) / c.std() if c.name[1].startswith('d') else c, axis=0)
    zscoredHeadMove.append(zscored)

zscoredDf = pd.concat(zscoredHeadMove, axis=1)

  0%|          | 0/23 [00:00<?, ?it/s]

In [39]:
zscoredDf
pd.concat(zscoredHeadMove, axis=0)

Head               LHand                         \
                             dMove    dAngle     dMove    dAngle InBox InWall   
Subject Block Condition                                                         
001     01    G.Assisted -0.815299 -0.810641 -0.722147 -0.081892   366    100   
              G.Ignored   0.879469  1.029891  0.485082  1.173601   802      1   
              G.Locked    0.796754  0.687362  0.841436  0.350542  3371     88   
        02    G.Assisted -1.012020 -1.022506 -0.917688 -1.013156     0      0   
              G.Ignored   1.048021  0.995360  1.306430  0.875070  2393    212   
...                            ...       ...       ...       ...   ...    ...   
023     01    G.Ignored  -0.714984 -0.703950  1.993835 -1.595728     0    104   
              G.Locked    1.684014  1.911566 -0.430241  0.736465     0      0   
        02    G.Assisted -0.513202 -0.234257 -0.300337  0.641758     0      0   
              G.Ignored  -0.643650 -0.480543 -0.655116 -0.911316     0      0   
              G.Locked    0.785863  0.224304 -0.564087  0.690516     0      0   

                             RHand                         
                             dMove    dAngle InBox InWall  
Subject Block Condition                                    
001     01    G.Assisted -0.723549 -0.670575     1      1  
              G.Ignored   0.850718  1.306072  2302      0  
              G.Locked   -0.092145  0.835718  5150     18  
        02    G.Assisted -0.797372 -0.818301     0      0  
              G.Ignored   1.568161  0.468311  2309      0  
...                            ...       ...   ...    ...  
023     01    G.Ignored   1.995747 -0.886537     0      0  
              G.Locked   -0.510023 -0.110637     0      0  
        02    G.Assisted -0.296315  0.600420     0      0  
              G.Ignored  -0.672453 -0.534959     0      0  
              G.Locked   -0.462502  1.715073     0      0  

[138 rows x 10 columns]

In [74]:
rot = df.loc['001', '00', 'G.Assisted', :]['LHand']['Rot']
# np.sqrt(((pos.shift() - pos)**2).sum(axis=1)).sum()
abs = (rot.shift() * rot).sum(axis=1, min_count=3).abs()
abs[abs > 1] = 1
abs[abs > 1 - 1e-5] = 0
abs[abs != 0] = np.arccos(abs[abs != 0]) * 2.0 * 180 / np.pi
abs[abs!=0]

total_angle = abs

TimeStamp
637933116582719837          NaN
637933116646909860    18.501647
637933116647679853     1.076972
637933116647979860     0.632982
637933116648179866     0.566952
                        ...    
637933117475297539     0.953853
637933117476301699     0.562235
637933117476517290     0.533279
637933117485368844     0.671101
637933117485732960     0.591693
Length: 391, dtype: float64